In [3]:
import numpy as np

from numpy.typing import NDArray
from fenics import (
    FunctionSpace, Function, TrialFunction, TestFunction,
    Constant, dot, grad, dx, solve, DirichletBC, UnitSquareMesh,
    assemble
)

In [4]:
def get_boundary_dofs(V: FunctionSpace) -> NDArray:
    """
    Get the indices of the boundary nodes.
    """
    def boundary(x, on_boundary):
        return on_boundary

    bc = DirichletBC(V, Constant(0.0), boundary)
    bc_dict = bc.get_boundary_values()
    bdofs = np.array(sorted(bc_dict.keys()), dtype=int)
    return bdofs


def assemble_S(V: FunctionSpace, sigma: float=1.0, k: float=1.0) -> NDArray:
    """
    Assemble the exact S matrix of the PDE.
    """
    u = TrialFunction(V)
    v = TestFunction(V)

    sigma = Constant(sigma)
    k = Constant(k)

    a = sigma * dot(grad(u), grad(v)) * dx + k * u * v * dx  # LHS
    m = u * v * dx  # RHS

    # Assemble A, M and S
    A = assemble(a).array()
    M = assemble(m).array()
    S = np.linalg.solve(A, M)  # S = A^{-1} @ M

    return S


def assemble_T(boundary_dofs: NDArray, N: int) -> NDArray:
    """
    Assemble the discrete trace matrix T.
    """
    boundary_dofs = np.array(boundary_dofs, dtype=int)
    Nb = len(boundary_dofs)

    T = np.zeros((Nb, N))

    for i, j in enumerate(boundary_dofs):
        T[i, j] = 1.0

    return T


def assemble_K(V_h: FunctionSpace) -> NDArray:
    """
    Assemble the exact discrete forward operator K = (T)(S)
    """
    N = V_h.dim()
    boundary_dofs = get_boundary_dofs(V_h)

    S = assemble_S(V_h)
    T = assemble_T(boundary_dofs, N)
    return T @ S

In [13]:
n = 64
mesh = UnitSquareMesh(n, n)
V_h = FunctionSpace(mesh, 'CG', 1)

N = V_h.dim()
N_b = len(get_boundary_dofs(V_h))
print(f"N_b={N_b}, N={N}")

N_b=256, N=4225


In [14]:
K = assemble_K(V_h)

In [15]:
K.shape

(256, 4225)